In [1]:
import urllib
import requests
import pandas as pd
import json
import base64

In [2]:
text = 'david :%&= silva'
urllib.parse.quote_plus(text)

'david+%3A%25%26%3D+silva'

In [3]:
query = 'artist:silva genre:pop tag:new'
url_query = urllib.parse.quote_plus(query)

In [4]:
def get_session_token(SessionID, SessionKey):
    url = 'https://accounts.spotify.com/api/token'
    data = {'grant_type':'client_credentials'}
    encoded_key = base64.b64encode(str(SessionID + ":" + SessionKey).\
                                   encode("ascii"))
    header = {'Authorization': 'Basic {}'.format(encoded_key.decode("ascii"))}
    response = requests.post(url,
                            data = data,
                            headers = header)
    print(response.status_code)
    return response.json()['access_token']

In [5]:
keys = pd.read_csv(r"../..\Dropbox\Kent\EMAT-32210_Data-in-EMAT\Spotify-Keys_9-12-23.txt")

In [6]:
# keys

In [7]:
access_token = get_session_token(keys['Client_ID'].iloc[0], keys['Client_Secret'].iloc[0])

200


In [8]:
access_token

'BQC8hoVlw_XHlmrf3aIcz3Q6PDFvmbGYMxg6uGSaKhO3-Donz7PKV6vRiB2ursSno2FYSVqS2V57RHU0amLbT_WLp53qQONyR0qSOR8K94wRecDku_g'

In [13]:
t_features_ep = 'https://api.spotify.com/v1/audio-features'
tracks_ep = 'https://api.spotify.com/v1/tracks'
ab_tracks_ep = 'https://api.spotify.com/v1/albums/{}/tracks'

In [18]:
def api_call(endpoint_url, api_header):
    response = requests.get(endpoint_url, headers = api_header)
    print(response.status_code)
    return response.json()

In [10]:
session_header = {'Authorization': 'Bearer {}'.format(access_token)}

In [11]:
sc_album_id = '3iPSVi54hsacKKl1xIR2eH'
cr_album_id = '0EiI8ylL0FmWWpgHVTsZjZ'

In [14]:
ab_tracks_ep.format(sc_album_id)

'https://api.spotify.com/v1/albums/3iPSVi54hsacKKl1xIR2eH/tracks'

In [21]:
sc_album_response = api_call(ab_tracks_ep.format(sc_album_id), session_header)
cr_album_response = api_call(ab_tracks_ep.format(cr_album_id), session_header)

200
200


In [28]:
sc_album_df = pd.DataFrame(sc_album_response['items'])
cr_album_df = pd.DataFrame(cr_album_response['items'])
# sc_album_df.head()

In [31]:
sc_track_ids = ','.join(sc_album_df['id'].to_list())
cr_track_ids = ','.join(cr_album_df['id'].to_list())

In [33]:
t_features_ep + '?ids={}'.format(sc_track_ids)

'https://api.spotify.com/v1/audio-features?ids=5G2f63n7IPVPPjfNIGih7Q,2tHwzyyOLoWSFqYNjeVMzj,102YUQbYmwdBXS7jwamI90,3iPIDAFybaoyqX7hvAfWkl,5oIVNm56t6OIf9ZjdEG3ud,1UHS8Rf6h5Ar3CDWRd3wjF,2HRqTpkrJO5ggZyyK6NPWz,5VhvD0AKRprEaFtPNKGBQR,0XkZmBCCcdMY0EPY8ij6Gb,21B4gaTWnTkuSh77iWEXdS,5il0jwWUlvgtIzWvzJi12z,09LrGvT9KsACH66RHYMDyR'

In [34]:
sc_track_features = api_call(t_features_ep + '?ids={}'.format(sc_track_ids),
                             session_header)
cr_track_features = api_call(t_features_ep + '?ids={}'.format(cr_track_ids),
                             session_header)

200
200


In [35]:
sc_track_info = api_call(tracks_ep + '?market=US&ids={}'.format(sc_track_ids),
                             session_header)
cr_track_info = api_call(tracks_ep + '?market=US&ids={}'.format(cr_track_ids),
                             session_header)

200
200


In [39]:
sc_features_df = pd.DataFrame(sc_track_features['audio_features'])
cr_features_df = pd.DataFrame(cr_track_features['audio_features'])
# sc_features_df.head()

In [43]:
sc_tracks_df = pd.DataFrame(sc_track_info['tracks'])
cr_tracks_df = pd.DataFrame(cr_track_info['tracks'])
sc_tracks_df.head()

,album,artists,disc_number,duration_ms,explicit,external_ids,external_urls,href,id,is_local,is_playable,name,popularity,track_number,type,uri
0,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,1,157279,False,{'isrc': 'USUM72404100'},{'spotify': 'https://open.spotify.com/track/5G...,https://api.spotify.com/v1/tracks/5G2f63n7IPVP...,5G2f63n7IPVPPjfNIGih7Q,False,True,Taste,96,1,track,spotify:track:5G2f63n7IPVPPjfNIGih7Q
1,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,1,186365,True,{'isrc': 'USUM72404101'},{'spotify': 'https://open.spotify.com/track/2t...,https://api.spotify.com/v1/tracks/2tHwzyyOLoWS...,2tHwzyyOLoWSFqYNjeVMzj,False,True,Please Please Please,84,2,track,spotify:track:2tHwzyyOLoWSFqYNjeVMzj
2,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,1,185265,True,{'isrc': 'USUM72404102'},{'spotify': 'https://open.spotify.com/track/10...,https://api.spotify.com/v1/tracks/102YUQbYmwdB...,102YUQbYmwdBXS7jwamI90,False,True,Good Graces,89,3,track,spotify:track:102YUQbYmwdBXS7jwamI90
3,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,1,218284,True,{'isrc': 'USUM72404104'},{'spotify': 'https://open.spotify.com/track/3i...,https://api.spotify.com/v1/tracks/3iPIDAFybaoy...,3iPIDAFybaoyqX7hvAfWkl,False,True,Sharpest Tool,86,4,track,spotify:track:3iPIDAFybaoyqX7hvAfWkl
4,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,1,164202,False,{'isrc': 'USUM72405667'},{'spotify': 'https://open.spotify.com/track/5o...,https://api.spotify.com/v1/tracks/5oIVNm56t6OI...,5oIVNm56t6OIf9ZjdEG3ud,False,True,Coincidence,86,5,track,spotify:track:5oIVNm56t6OIf9ZjdEG3ud
